# Data Cleaning

### Configuración espacio de trabajo

In [38]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from repo_funciones import *


In [39]:
import re

In [40]:
#%pip install openpyxl

## Archivo 1: Censo de hogares y viviendas 2020

In [56]:
censo=pd.read_excel('../DATA/censo_viv_CDMX.xlsx', sheet_name=10, skiprows=4, header=1) 


In [57]:
censo.head()

,Entidad federativa,Demarcación territorial,Disponibilidad y ámbito de agua entubada,Fuente de abastecimiento u obtención de agua,Viviendas particulares habitadas,Unnamed: 5,Unnamed: 6,Unnamed: 7,Ocupantes de viviendas particulares habitadas,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,Total,Disponibilidad de drenaje,NaN,NaN,Total,Disponibilidad de drenaje,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Disponen de drenaje,No disponen de drenaje,No especificado,NaN,Disponen de drenaje,No disponen de drenaje,No especificado
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09 Ciudad de México,Total,Total,Total,2752169,2744306,4207,3656,9147377,9123074,13509,10794
4,09 Ciudad de México,Total,Disponen de agua entubada,Total,2719720,2716974,2536,210,9023868,9015092,8100,676


In [58]:
censo.drop(columns=['Entidad federativa', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Viviendas particulares habitadas'], inplace=True)

In [59]:
censo.head()

,Demarcación territorial,Disponibilidad y ámbito de agua entubada,Fuente de abastecimiento u obtención de agua,Ocupantes de viviendas particulares habitadas
0,NaN,NaN,NaN,Total
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,Total,Total,Total,9147377
4,Total,Disponen de agua entubada,Total,9023868


In [60]:
censo= censo.drop([0,1,2]).reset_index(drop=True)

In [64]:
censo.loc[0:35, 'Demarcación territorial'] = 'CDMX (total)' #venian las primeras 36 entradas como 'total' haciendo referencia al total Ciudad de Mexico

In [62]:
#Las alcaldías vienen con su codigo territorial antes del nombre de la alcaldía. Usamos regex para quitar los valores numericos y dejar solo texto.

texto = re.compile(r'^\d+\s*(.*)')
censo['Demarcación territorial'] = censo['Demarcación territorial'].str.extract(texto, expand=False)

In [65]:
col_unique_counts(censo, 'Demarcación territorial') #vemos que el conteo de valores unicos por alcaldía no son iguales. 
#esto significa que en otras columnas no van a tener las mismas entradas. Ahora que cambiemos de columna vemos como lo solucionamos.

,Demarcación territorial,Count
0,CDMX (total),36
1,Iztapalapa,36
2,Cuajimalpa de Morelos,36
3,Álvaro Obregón,36
4,Venustiano Carranza,35
5,Tlalpan,35
6,Tláhuac,35
7,Milpa Alta,35
8,La Magdalena Contreras,35
9,Xochimilco,35


In [53]:
#censo.head(100) #hay muchas filas que son subtotales de otras filas. Esto puede confundir el analisis y vamos a ir dejando mas valores unicos posibles. 
#por ejemplo, "disponen de agua entubada" es el subtotal de "Disponen de agua entubada dentro de la vivienda" y "Disponen de agua entubada solo en el patio o terreno"
#Como este proyecto va dirigido a los mas necesitados, nos nos importa tanto esta distincción. Nos quedamos unicamente con la fila de subtotales. 

In [67]:
censo= censo.drop(index=range(0, 36)).reset_index(drop=True) #primeras 36 filas que corresponden al subtotal de todas las alcadías. 

In [68]:
censo.head()

,Demarcación territorial,Disponibilidad y ámbito de agua entubada,Fuente de abastecimiento u obtención de agua,Ocupantes de viviendas particulares habitadas
0,Álvaro Obregón,Total,Total,756256
1,Álvaro Obregón,Disponen de agua entubada,Total,753699
2,Álvaro Obregón,Disponen de agua entubada,Del servicio público de agua,744504
3,Álvaro Obregón,Disponen de agua entubada,De un pozo comunitario,5020
4,Álvaro Obregón,Disponen de agua entubada,De un pozo particular,170


In [70]:
get_unique_dtypes(censo, 'Disponibilidad y ámbito de agua entubada')

array([<class 'str'>, <class 'float'>], dtype=object)

In [71]:
col_unique_counts(censo, 'Disponibilidad y ámbito de agua entubada')

,Disponibilidad y ámbito de agua entubada,Count
0,Disponen de agua entubada,144
1,Disponen de agua entubada dentro de la vivienda,144
2,Disponen de agua entubada solo en el patio o t...,127
3,No disponen de agua entubada,105
4,Total,16
5,No especificado,16


In [72]:
float_rows = censo[censo['Disponibilidad y ámbito de agua entubada'].apply(lambda x: isinstance(x, float))]


In [73]:
float_rows

,Demarcación territorial,Disponibilidad y ámbito de agua entubada,Fuente de abastecimiento u obtención de agua,Ocupantes de viviendas particulares habitadas
552,NaN,NaN,NaN,NaN
553,NaN,NaN,NaN,NaN


In [74]:
censo.tail() 

,Demarcación territorial,Disponibilidad y ámbito de agua entubada,Fuente de abastecimiento u obtención de agua,Ocupantes de viviendas particulares habitadas
549,Xochimilco,No disponen de agua entubada,La trae una pipa,26942
550,Xochimilco,No disponen de agua entubada,La recolectan de la lluvia,234
551,Xochimilco,No especificado,Total,211
552,NaN,NaN,NaN,NaN
553,NaN,NaN,NaN,NaN


In [75]:
censo= censo.drop([552,553]) #valores nulos del excel que no me había fijado estaban al final del todo del archivo excel. 

In [76]:
censo = censo[~censo['Disponibilidad y ámbito de agua entubada'].str.contains('Disponen de agua entubada dentro de la vivienda|Disponen de agua entubada solo en el patio o terreno')]
censo = censo.reset_index(drop=True)
#este codigo es para quitar las filas que separaban disponibilidad de agua en vivienda y en terreno. Nos quedamos unicamente con el subtotal que ya venía en la tabla.
#Este codigo hace psicología inversa con ~. Busca todas las filas dentro de la columna que contengan lo que queremos borrar y la negación hace que el DF se quede con todo lo que no es lo filtrado.

In [80]:
censo.head(10) #vemos que nos faltan dos tipos de sub totales en el DF. Los que agrupan el total de disponibilidad(o no) de agua en la segunda columna 
# y los que agregan por alcadía. Ambos contienen 'total' en la columna fuente. 

,Demarcación territorial,Disponibilidad y ámbito de agua entubada,Fuente de abastecimiento u obtención de agua,Ocupantes de viviendas particulares habitadas
0,Álvaro Obregón,Total,Total,756256
1,Álvaro Obregón,Disponen de agua entubada,Total,753699
2,Álvaro Obregón,Disponen de agua entubada,Del servicio público de agua,744504
3,Álvaro Obregón,Disponen de agua entubada,De un pozo comunitario,5020
4,Álvaro Obregón,Disponen de agua entubada,De un pozo particular,170
5,Álvaro Obregón,Disponen de agua entubada,De una pipa,2133
6,Álvaro Obregón,Disponen de agua entubada,De otra vivienda,637
7,Álvaro Obregón,Disponen de agua entubada,De la lluvia,26
8,Álvaro Obregón,Disponen de agua entubada,De otro lugar,308
9,Álvaro Obregón,Disponen de agua entubada,No especificado,901


In [81]:
censo = censo[~censo['Fuente de abastecimiento u obtención de agua'].str.contains('Total')]

In [84]:
censo.head(10)

,Demarcación territorial,Disponibilidad y ámbito de agua entubada,Fuente de abastecimiento u obtención de agua,Ocupantes de viviendas particulares habitadas
2,Álvaro Obregón,Disponen de agua entubada,Del servicio público de agua,744504
3,Álvaro Obregón,Disponen de agua entubada,De un pozo comunitario,5020
4,Álvaro Obregón,Disponen de agua entubada,De un pozo particular,170
5,Álvaro Obregón,Disponen de agua entubada,De una pipa,2133
6,Álvaro Obregón,Disponen de agua entubada,De otra vivienda,637
7,Álvaro Obregón,Disponen de agua entubada,De la lluvia,26
8,Álvaro Obregón,Disponen de agua entubada,De otro lugar,308
9,Álvaro Obregón,Disponen de agua entubada,No especificado,901
11,Álvaro Obregón,No disponen de agua entubada,Acarreada de un pozo,19
12,Álvaro Obregón,No disponen de agua entubada,Acarreada de llave comunitaria,412


In [86]:
col_names= ['alcaldia', 'disponibilidad', 'fuente', 'poblacion']
censo.columns=col_names

In [87]:
censo.head()

,alcaldia,disponibilidad,fuente,poblacion
2,Álvaro Obregón,Disponen de agua entubada,Del servicio público de agua,744504
3,Álvaro Obregón,Disponen de agua entubada,De un pozo comunitario,5020
4,Álvaro Obregón,Disponen de agua entubada,De un pozo particular,170
5,Álvaro Obregón,Disponen de agua entubada,De una pipa,2133
6,Álvaro Obregón,Disponen de agua entubada,De otra vivienda,637


In [88]:
data = {'alcaldia_id': ['002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017'],
        'alcaldia': ['Azcapotzalco', 'Coyoacán', 'Cuajimalpa de Morelos', 'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras', 'Milpa Alta', 'Álvaro Obregón', 'Tláhuac', 'Tlalpan', 'Xochimilco', 'Benito Juárez', 'Cuauhtémoc', 'Miguel Hidalgo', 'Venustiano Carranza']}

alcaldias = pd.DataFrame(data) #recuperamos los codigos territoriales que con tanta passión nos cargamos antes. 

#aparte de que necesitaremos una tabla con esta info para SQL, necesitamos agregar el ID a la tabla censo. 

In [90]:
alc = pd.merge(censo, alcaldias[['alcaldia', 'alcaldia_id']], on='alcaldia', how='left')
censo['alcaldia_id'] = alc['alcaldia_id']

In [106]:
nancols(censo) 

alcaldia_id    50
dtype: int64

In [117]:
#censo[censo['alcaldia_id'].isna()]
#las ultimas 50 filas parece que no jaló bien la formula. Lo remplazamos manualmente. 

In [115]:
censo.loc[censo['alcaldia']=='Tláhuac', 'alcaldia_id']='011'
censo.loc[censo['alcaldia']=='Tlalpan', 'alcaldia_id']='012'
censo.loc[censo['alcaldia']=='Venustiano Carranza', 'alcaldia_id']='017'
censo.loc[censo['alcaldia']=='Xochimilco', 'alcaldia_id']='013'

In [118]:
censo.to_csv('../Data/censo_clean.csv', index=False) 
alcaldias.to_csv('../DATA/alcaldias.csv')

### Archivo 2: Proyectos de captación de agua de lluvia CDMX 2022

In [149]:
SCALL=pd.read_csv('../DATA/captacion_agua_lluvias_2022.csv')

In [150]:
SCALL.head()

,_id,expediente,pueblo,scall_col,territorial,fecha_instalacion,capacidad,alcaldia,colonia_datos,latitud,longitud
0,1,1-MEMEMA-MIA-22,SAN ANTONIO TECOMITL,BARRIO XALTIPAC,NO APLICA,2022-05-02T00:00:00,2500,MILPA ALTA,SAN ANTONIO TECOMITL (PBLO),19.219028,-98.991283
1,2,2-RUCOAL-MIA-22,SAN ANTONIO TECOMITL,BARRIO TENANTITLA,NO APLICA,2022-05-07T00:00:00,2500,MILPA ALTA,SAN ANTONIO TECOMITL (PBLO),19.219880,-99.002227
2,3,3-GAALRA-MIA-22,SAN ANTONIO TECOMITL,BARRIO XALTIPAC,NO APLICA,2022-07-21T00:00:00,2500,MILPA ALTA,SAN ANTONIO TECOMITL (PBLO),19.218343,-98.997507
3,4,4-BLROAN-MIA-22,SAN ANTONIO TECOMITL,BARRIO TENANTITLA,NO APLICA,2022-05-02T00:00:00,2500,MILPA ALTA,SAN ANTONIO TECOMITL (PBLO),19.219277,-98.996269
4,5,8-BALOMO-MIA-22,SAN ANTONIO TECOMITL,BARRIO TENANTITLA,NO APLICA,2022-05-19T00:00:00,2500,MILPA ALTA,SAN ANTONIO TECOMITL (PBLO),19.222071,-98.997760


In [151]:
SCALL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16971 entries, 0 to 16970
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _id                16971 non-null  int64  
 1   expediente         16971 non-null  object 
 2   pueblo             16971 non-null  object 
 3   scall_col          16971 non-null  object 
 4   territorial        16971 non-null  object 
 5   fecha_instalacion  16971 non-null  object 
 6   capacidad          16971 non-null  int64  
 7   alcaldia           15943 non-null  object 
 8   colonia_datos      15943 non-null  object 
 9   latitud            16971 non-null  float64
 10  longitud           16971 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1.4+ MB


In [152]:
nancols(SCALL) #Como nuestro analisis va a ser a nivel alcaldías tenemos que limpiar esa columna de nulos. 
#la colonia la descartaremos. 

alcaldia         1028
colonia_datos    1028
dtype: int64

In [153]:
SCALL.loc[SCALL['pueblo'] == 'PBO. SANTIAGO TUYLYEHUALCO', 'alcaldia']='Xochimilco'
SCALL.loc[SCALL['pueblo'] == 'PUEBLO SAN LORENZO TLACOYUCAN', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'SAN PABLO OZTOTEPEC', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'VILLA MILPA ALTA', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['_id'] == 4275, 'alcaldia']='Tlalpan' #no tenia pueblo, lo miré por las coordenadas
SCALL.loc[SCALL['pueblo'] == 'SAN PEDRO ATOCPAN', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'PUEBLO SAN FRANCISCO TECOXPA', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'PUEBLO SAN LORENZO TLACOYUCAN', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'SANTA ANA TLACOTENCO', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'PUEBLO SAN BARTOLOME XICOMULCO', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'PUEBLO SAN SALVADOR CUAUHTENCO', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['pueblo'] == 'SAN JUAN IXTAYOPAN', 'alcaldia']='Tláhuac'
SCALL.loc[SCALL['scall_col'] == 'PUEBLO PARRES EL GUARDA', 'alcaldia']='Tlalpan'
SCALL.loc[SCALL['pueblo'] == 'PUEBLO SANTO TOMAS AJUSCO', 'alcaldia']='Tlalpan'
SCALL.loc[SCALL['scall_col'] == 'ATOCPA', 'alcaldia']='Tlalpan'
SCALL.loc[SCALL['scall_col'] == 'TLALMILLE', 'alcaldia']='Tlalpan'
SCALL.loc[SCALL['pueblo'] == 'PUEBLO PARRES EL GUARDA', 'alcaldia']='Tlalpan'
SCALL.loc[SCALL['scall_col'] == 'CUCHILLA DEL TESORO', 'alcaldia']='Gustavo A. Madero'
SCALL.loc[SCALL['pueblo'] == 'SAN ANTONIO TECOMITL', 'alcaldia']='Milpa Alta'
SCALL.loc[SCALL['scall_col'] == 'VISTA HERMOSA', 'alcaldia']='Gustavo A. Madero'
SCALL.loc[SCALL['scall_col'] == 'SAN FELIPE DE JESUS', 'alcaldia']='Gustavo A. Madero'

In [154]:
nancols(SCALL) #ACUERDATE DE TITLE PARA ALCALDIAS Y AÑADIR ALCADIAid


colonia_datos    1028
dtype: int64

In [155]:
SCALL.drop(columns=['_id', 'expediente', 'pueblo', 'scall_col', 'territorial', 'fecha_instalacion', 'colonia_datos' ], inplace=True)

In [156]:
SCALL['alcaldia'] = SCALL['alcaldia'].str.title()

In [157]:
SCALL.head()

,capacidad,alcaldia,latitud,longitud
0,2500,Milpa Alta,19.219028,-98.991283
1,2500,Milpa Alta,19.219880,-99.002227
2,2500,Milpa Alta,19.218343,-98.997507
3,2500,Milpa Alta,19.219277,-98.996269
4,2500,Milpa Alta,19.222071,-98.997760


In [158]:
alc2 = pd.merge(SCALL, alcaldias[['alcaldia', 'alcaldia_id']], on='alcaldia', how='left')
SCALL['alcaldia_id'] = alc2['alcaldia_id']

In [159]:
nancols(SCALL)

alcaldia_id    1982
dtype: int64

In [161]:
#SCALL[SCALL['alcaldia_id'].isna()] el merge de antes no funcionó porque Tlahuac venía sin tilde en la primera A.

In [162]:
SCALL.loc[SCALL['alcaldia'] == 'Tlahuac', 'alcaldia']='Tláhuac'
SCALL.loc[SCALL['alcaldia'] == 'Tláhuac', 'alcaldia_id']='011'

In [164]:
SCALL.to_csv('../DATA/captacion_clean.csv')